In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('./titanic.csv')

print("Primeiras 5 linhas do dataset:")
df.head()

In [ ]:
# infos sobre o dataset
print("\nInformações sobre o dataset:")
df.info()

print("\nEstatísticas descritivas:")
df.describe()

print("\nValores ausentes por coluna:")
df.isnull().sum()

In [ ]:
# visualização de distribuição e sobreviventes
plt.figure(figsize=(10, 6))
sns.countplot(x='Survived', data=df, palette='Blues')
plt.title('Distribuição de Sobreviventes', fontsize=16)
plt.xlabel('Sobreviveu (0=Não, 1=Sim)', fontsize=14)
plt.ylabel('Contagem', fontsize=14)
plt.xticks([0, 1], ['Não Sobreviveu', 'Sobreviveu'])
plt.show()

sobreviventes = df['Survived'].value_counts(normalize=True) * 100
print(f"Percentual de sobreviventes: {sobreviventes[1]:.2f}%")
print(f"Percentual de não sobreviventes: {sobreviventes[0]:.2f}%")

In [ ]:
# sobrevivência por classe
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x='Pclass', hue='Survived', data=df, palette='viridis')
plt.title('Sobrevivência por Classe', fontsize=14)
plt.xlabel('Classe', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.legend(title='Sobreviveu', labels=['Não', 'Sim'])

# sobrevivência por sexo
plt.subplot(1, 2, 2)
sns.countplot(x='Sex', hue='Survived', data=df, palette='viridis')
plt.title('Sobrevivência por Sexo', fontsize=14)
plt.xlabel('Sexo', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.legend(title='Sobreviveu', labels=['Não', 'Sim'])
plt.tight_layout()
plt.show()

In [ ]:
# distribuição de idade
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(data=df, x='Age', hue='Survived', kde=True, palette='coolwarm')
plt.title('Distribuição de Idade por Sobrevivência', fontsize=14)
plt.xlabel('Idade', fontsize=12)
plt.ylabel('Contagem', fontsize=12)

In [ ]:
# criando feature de tamanho da família
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

plt.figure(figsize=(12, 6))
sns.countplot(x='FamilySize', hue='Survived', data=df, palette='muted')
plt.title('Sobrevivência por Tamanho da Família', fontsize=16)
plt.xlabel('Tamanho da Família', fontsize=14)
plt.ylabel('Contagem', fontsize=14)
plt.legend(title='Sobreviveu', labels=['Não', 'Sim'])
plt.show()

In [ ]:
# matriz de correlação
numeric_cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']
correlation = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5, mask=mask)
plt.title('Matriz de Correlação', fontsize=16)
plt.show()

In [ ]:
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

# Extrair títulos
df['Title'] = df['Name'].apply(get_title)

# Agrupar títulos menos comuns
title_mapping = {
    "Mr": "Mr",
    "Miss": "Miss",
    "Mrs": "Mrs",
    "Master": "Master",
    "Dr": "Other",
    "Rev": "Other",
    "Col": "Other",
    "Major": "Other",
    "Mlle": "Miss",
    "Countess": "Other",
    "Ms": "Miss",
    "Lady": "Other",
    "Jonkheer": "Other",
    "Don": "Other",
    "Dona": "Other",
    "Mme": "Mrs",
    "Capt": "Other",
    "Sir": "Other"
}

df['Title'] = df['Title'].map(lambda x: title_mapping.get(x, 'Other'))

df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

embarked_dummies = pd.get_dummies(df['Embarked'], prefix='Embarked')
title_dummies = pd.get_dummies(df['Title'], prefix='Title')

df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

age_imputer = SimpleImputer(strategy='median')
df['Age'] = age_imputer.fit_transform(df[['Age']])

df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

df['Fare'].fillna(df['Fare'].median(), inplace=True)

df = pd.concat([df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone']], 
                embarked_dummies, title_dummies], axis=1)

print("Dataset após pré-processamento:")
df.head()


In [ ]:
# definir targer
X = df.drop('Survived', axis=1)
y = df['Survived']

# dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")

In [ ]:
# Logistic Regression Model
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train_scaled, y_train)

y_pred_log = log_model.predict(X_test_scaled)

print("=== Regressão Logística ===")
conf_matrix_log = confusion_matrix(y_test, y_pred_log)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_log))


plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_log, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Não Sobreviveu', 'Sobreviveu'],
            yticklabels=['Não Sobreviveu', 'Sobreviveu'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()


with open('logistic_regression_titanic.pickle', 'wb') as f:
    pickle.dump(log_model, f)

In [ ]:
# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train) 

y_pred_rf = rf_model.predict(X_test)

print("=== Random Forest ===")
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_rf, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Não Sobreviveu', 'Sobreviveu'],
            yticklabels=['Não Sobreviveu', 'Sobreviveu'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Random Forest')
plt.show()

feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(10), palette='viridis')
plt.title('Top 10 Features Mais Importantes - Random Forest', fontsize=16)
plt.xlabel('Importância', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()

with open('random_forest_titanic.pickle', 'wb') as f:
    pickle.dump(rf_model, f)

In [ ]:
# Support Vector Machine model
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)  # SVM funciona melhor com dados normalizados

y_pred_svm = svm_model.predict(X_test_scaled)

print("=== Support Vector Machine ===")

conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=['Não Sobreviveu', 'Sobreviveu'],
            yticklabels=['Não Sobreviveu', 'Sobreviveu'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - SVM')
plt.show()

with open('svm_titanic.pickle', 'wb') as f:
    pickle.dump(svm_model, f)

In [ ]:
models = ['Regressão Logística', 'Random Forest', 'SVM']
accuracy = [
    classification_report(y_test, y_pred_log, output_dict=True)['accuracy'],
    classification_report(y_test, y_pred_rf, output_dict=True)['accuracy'],
    classification_report(y_test, y_pred_svm, output_dict=True)['accuracy']
]

plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=accuracy, palette='viridis')
plt.title('Comparação de Acurácia entre os Modelos', fontsize=16)
plt.xlabel('Modelo', fontsize=14)
plt.ylabel('Acurácia', fontsize=14)
plt.ylim(0.7, 0.9)
plt.tight_layout()
plt.show()

In [ ]:
with open('./logistic_regression_titanic.pickle', 'rb') as f:
    loaded_log_model = pickle.load(f)

with open('./random_forest_titanic.pickle', 'rb') as f:
    loaded_rf_model = pickle.load(f)

with open('./svm_titanic.pickle', 'rb') as f:
    loaded_svm_model = pickle.load(f)

loaded_log_pred = loaded_log_model.predict(X_test_scaled)
loaded_rf_pred = loaded_rf_model.predict(X_test)
loaded_svm_pred = loaded_svm_model.predict(X_test_scaled)

print("Regressão Logística - Previsões iguais:", np.array_equal(y_pred_log, loaded_log_pred))
print("Random Forest - Previsões iguais:", np.array_equal(y_pred_rf, loaded_rf_pred))
print("SVM - Previsões iguais:", np.array_equal(y_pred_svm, loaded_svm_pred))